In [1]:
import pdfplumber
import PyPDF2
import ast
import tiktoken
import requests
import re
import os,glob
import time
from random import sample
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def count_tokens(text):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(text))
    return num_tokens

In [3]:
def get_txt_from_pdf(pdf_files,filter_ref = False, combine=False):
    data = []
    pdf_content_dict = {}
    for pdf in pdf_files:
        all_pdf_txt = ''
        print(pdf)
        with open(pdf, 'rb') as pdf_content:
            pdf_reader = PyPDF2.PdfReader(pdf_content)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                page_text = page.extract_text()
                all_pdf_txt += page_text
            all_pdf_txt = re.split('\n',all_pdf_txt)
            all_pdf_txt = [txt1 for txt in all_pdf_txt for txt1 in re.split('\n \n',txt)]
            for i, para_part in enumerate(all_pdf_txt):
                data.append({
                    'file name': pdf,
                    'paragraph number': i+1,
                    'content': para_part,
                    'tokens': count_tokens(para_part)
                })
    df = pd.DataFrame(data)
    return df

In [6]:
folder_name = r'E:\liuming\cage-GPT\papers\*\*.pdf'
#folder_name = r'E:\liuming\cage-GPT\additional papers\finished\*.pdf'
all_papers = glob.glob(folder_name)
df = get_txt_from_pdf(all_papers)

In [5]:
df

""


In [8]:
all_paper_list = list(pdf_content_dict.keys())

In [18]:
keys = pdf_content_dict.keys()

In [7]:
start_index = 0
all_groups = []
data = []
for i in range(len(df)):
    pdf = df['file name'][i]
    content_split = df['content'][i].split()
    if content_split != []:
        content_end = content_split[-1][-1]
        if content_end == '.' or content_end == '*':
            all_groups.append(range(start_index,i+1))
            new_content = ''
            for j in range(start_index,i+1):
                new_content += df['content'][j]
            data.append({
                        'file name': pdf,
                        'content': new_content,
                        'tokens': count_tokens(new_content)
                        })
            start_index = i+1
new_df = pd.DataFrame(data)

In [8]:
new_df = pd.DataFrame(data)
new_df

In [8]:
new_df['new_content'] = ''
new_df['part'] = ''

In [7]:
index = 1
part_num = 1
txt = ''
all_paper = {}
for i in range(len(new_df)):
    content = new_df.loc[i,'content']
    filename1 = new_df.loc[i,'file name']
    if i>0:
        filename2 = new_df.loc[i-1,'file name']
        if  filename1!=filename2  or i == len(new_df)-1:
            index = 1
            all_paper[filename2+'part'+str(part_num)] = txt
            txt = ''
            part_num = 1
        if  token_num > 3300-count_tokens(content) or index >= 20:
            index = 1
            all_paper[filename1+'part'+str(part_num)] = txt
            part_num += 1
            txt = ''
    txt += str(index)+'、'+content+'\n \n'
    token_num = count_tokens(txt)
    new_df.loc[i,'index'] = index
    new_df.loc[i,'new_content'] = str(index)+'、'+content
    new_df.loc[i,'part'] = filename1+'part'+str(part_num)
    index += 1

In [8]:
new_df.to_excel(r"E:\liuming\cage-GPT\all_labelled_text.xlsx")

In [12]:
from openai import OpenAI
api_key = "***"
organization = '***'
client = OpenAI(api_key=api_key,organization=organization)

In [13]:
with open('./classification.txt','r',encoding='utf-8') as f:
    prompt = f.read()
# with open('./paper_text.txt','r',encoding='utf-8') as f:
#     text = f.read()
assistant = "You are a classifier and you have learned the examples given in the system, \
now you are given a text containing many numbered paragraphs separated by a blank line, \
please output the category of each paragraph.Each category must be unambiguous, separated by a semicolon. \
If there is more than one category in a paragraph, output the main category.\
Write only the category in your answer, not the original text."

In [18]:
all_answers = {}
data = []
for index, key in enumerate(all_paper_list):
    text = all_paper[key]
    try:
        response = client.chat.completions.create(
                           model='gpt-4',
                            messages=[      {"role": "assistant", "content": assistant},
                                            {"role": "system", "content": prompt},
                                            {"role": "user", "content": text}
                                        ]
                        )
        answers = response.choices[0].message.content
        all_answers[key] = answers
        print(index+1,key)
        data.append({
                        'file name': key,
                        'content': text,
                        'categories': answers
                        })
    except Exception as e:
        print(f"Error: {str(e)}")

In [19]:
data = []
all_answers_keys = list(all_answers.keys())
for index, key in enumerate(all_paper_list):
    if key not in all_answers_keys:
        text = all_paper[key]
        data.append({'file name': key,'content': text})
long_text_df = pd.DataFrame(data)

In [22]:
data = []
for index, key in enumerate(all_paper.keys()):
    if key in all_answers.keys():
        text = all_paper[key].split('\n \n')
        answers = re.split('\n|;',all_answers[key])
        answers = [answer for answer in answers if answer != '']
        if len(text) > len(answers):
            for i in range(len(answers)):
                para = text[i]
                answer = answers[i]
                #answer = re.split('.|:',answer)[-1]
                data.append({
                            'file name': key,
                            'content': para,
                            'categories': answer
                            })
        else:
            for i in range(len(text)):
                para = text[i]
                answer = answers[i]
                #answer = re.split('.|:',answer)[-1]
                data.append({
                            'file name': key,
                            'content': para,
                            'categories': answer
                            })
class_df = pd.DataFrame(data)